# ***Get Data***

* Import Library

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from annoy import AnnoyIndex

* Get data

In [2]:
df = pd.read_csv('../data/test.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,NaN,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


* Lakukan pengecekan data yang missing

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [6]:
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns
df_num = df[numeric_features].drop('Id', axis=1)

original_columns = list(df_num.columns)

print("--- Jumlah Missing Values SEBELUM Imputasi ---")
missing_values_before = df_num.isnull().sum()
print(missing_values_before[missing_values_before > 0].sort_values(ascending=False))

--- Jumlah Missing Values SEBELUM Imputasi ---
LotFrontage     227
GarageYrBlt      78
MasVnrArea       15
BsmtHalfBath      2
BsmtFullBath      2
BsmtFinSF1        1
BsmtFinSF2        1
TotalBsmtSF       1
BsmtUnfSF         1
GarageCars        1
GarageArea        1
dtype: int64


* Lakukan imputasi pada data

In [7]:
imputer = SimpleImputer(strategy='mean')
df_imputed = imputer.fit_transform(df_num)

df_imputed = pd.DataFrame(df_imputed, columns=original_columns)
print("Data berhasil di-imputasi.")
print(f"Jumlah NaN setelah imputasi: {df_imputed.isnull().sum().sum()}\n")

Data berhasil di-imputasi.
Jumlah NaN setelah imputasi: 0



In [8]:
df_imputed['TotalArea'] = df_imputed['GrLivArea'] + df_imputed['TotalBsmtSF']
print(f"Bentuk data setelah menambah fitur baru: {df_imputed.shape}\n")

Bentuk data setelah menambah fitur baru: (1459, 37)



In [9]:
selector = VarianceThreshold(threshold=0.0)
columns_before_variance_check = df_imputed.columns
df_variant = selector.fit_transform(df_imputed)

dropped_columns = columns_before_variance_check[~selector.get_support()]
if len(dropped_columns) > 0:
    print(f"Membuang {len(dropped_columns)} kolom dengan variasi nol:")
    print(list(dropped_columns))
else:
    print("Tidak ada kolom dengan variasi nol yang dibuang.")
print(f"Bentuk data setelah membuang variasi nol: {df_variant.shape}\n")

Tidak ada kolom dengan variasi nol yang dibuang.
Bentuk data setelah membuang variasi nol: (1459, 37)



* Lakukan pengecekan kembali untuk missing data

In [12]:
scaler = StandardScaler() #
X_scaled = scaler.fit_transform(df_variant)

print("--- Proses Preprocessing Selesai ---")
print(f"Bentuk akhir X_scaled (siap clustering): {X_scaled.shape}")
print(f"Jumlah NaN di data akhir (Cek final): {np.isnan(X_scaled).sum()}")

--- Proses Preprocessing Selesai ---
Bentuk akhir X_scaled (siap clustering): (1459, 37)
Jumlah NaN di data akhir (Cek final): 0


# ***Clustering***

* Melakukan penyiapan untuk K-Means & DBSCAN

In [14]:
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
labels_kmeans = kmeans.fit_predict(X_scaled)

print(f"Contoh 10 label pertama: {labels_kmeans[:10]}")

dbscan = DBSCAN(eps=3.2, min_samples=10)
labels_dbscan = dbscan.fit_predict(X_scaled)

print(f"Contoh 10 label pertama: {labels_dbscan[:10]}")

Contoh 10 label pertama: [4 4 3 3 2 3 2 3 2 4]
Contoh 10 label pertama: [ 2 -1  0  0 -1  0 -1  0 -1  2]


* Pengh